In [2]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [3]:
# Create engine using the `Tesla.sqlite` database file
engine = create_engine("sqlite:///Tesla.sqlite")

In [4]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

supercharge_locations
-----------
Supercharger TEXT
Street Address TEXT
City TEXT
State TEXT
Country TEXT
Stalls BIGINT
kW FLOAT
Elev(m) BIGINT
Open Date DATETIME
Latitude FLOAT
Longitude FLOAT



In [5]:
#this is for the basic where the locations are 
query = f"""
        SELECT City, State, Country, COUNT(*) AS Location_Count
        FROM supercharge_locations
        GROUP BY City, State, Country
        ORDER BY Location_Count DESC;
        """
gen_loc = pd.read_sql(text(query), con=engine)
gen_loc.head(50)

,City,State,Country,Location_Count
0,Shanghai,Shanghai,China,171
1,Beijing,Beijing,China,124
2,Hangzhou,Zhejiang,China,76
3,Shenzhen,Guangdong,China,58
4,Guangzhou,Guangdong,China,56
5,Chengdu,Sichuan,China,46
6,Chongqing,Chongqing,China,42
7,Hong Kong,Hong Kong,China,38
8,Ningbo,Zhejiang,China,29
9,Tianjin,Tianjin,China,29


In [6]:
#this is for the average stalls/kw per city in a given country
query= f'''
    SELECT State, Country, AVG(Stalls) AS Average_Stalls, AVG(kW) AS Average_kW
    FROM supercharge_locations
    GROUP BY State
    ORDER BY Average_Stalls DESC, Average_kW DESC
    '''
avg_kw = pd.read_sql(text(query), con=engine)
avg_kw.head(50)

,State,Country,Average_Stalls,Average_kW
0,Agder,Norway,22.666667,200.000000
1,Viken,Norway,22.631579,185.789474
2,Utrecht,Netherlands,22.400000,230.000000
3,Halland,Sweden,21.500000,200.000000
4,Östergötland,Sweden,21.500000,200.000000
5,Overijssel,Netherlands,20.000000,250.000000
6,Bohuslän,Sweden,20.000000,210.000000
7,Thuringia,Germany,19.500000,225.000000
8,Innlandet,Norway,19.333333,211.111111
9,Oslo,Norway,19.000000,250.000000


In [7]:
#this is going to be locations by countries
query = f"""
        SELECT Country, COUNT(*) AS Location_Count
        FROM supercharge_locations
        GROUP BY Country
        ORDER BY Location_Count DESC;
        """
country_loc = pd.read_sql(text(query), con=engine)
country_loc.head(50)

,Country,Location_Count
0,USA,1791
1,China,1701
2,Canada,186
3,Germany,170
4,France,145
5,South Korea,137
6,United Kingdom,117
7,Norway,106
8,Taiwan,76
9,Sweden,73


In [8]:
#this is going to be states in a given country 

In [9]:
#this is going to be the average stalls and kw per country 

In [10]:
#this is going to be the average stalls and kw per state in a country 

In [11]:
#different locations and elevations 

In [12]:
#elevations and average stalls and kw

In [13]:
#map code?